In [8]:
# -*- coding: utf-8 -*-

# author: s0nnet
# time: 2016-11-08
# desc: tb_order_dao


'''
# 订单表
CREATE TABLE `tb_order` (
`oid` int(10) NOT NULL AUTO_INCREMENT,
`cid` int(10) NOT NULL,
`name` varchar(50) NOT NULL,
`otype` varchar(10) NOT NULL DEFAULT 'A',
`order_tm` datetime NOT NULL,
`start_tm` date NOT NULL,
`end_tm` date NOT NULL,
`amount` float NOT NULL,
`cash` float NOT NULL,
`status` enum('normal','stop') DEFAULT 'normal',
`remark` text NOT NULL,
`insert_tm` datetime NOT NULL,
PRIMARY KEY (`oid`),
KEY `cid` (`cid`),
CONSTRAINT `tb_order_ibfk_1` FOREIGN KEY (`cid`) REFERENCES `tb_custom` (`cid`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8;
'''



if __name__ == "__main__":
    import sys
    sys.path.append("..")
    sys.path.append("../base")

from hjs_cfg import *
from bs_util import *
from bs_database_pid import *


class HjsOrderDao:

    @staticmethod
    def insert_node(cId, name, otype, order_tm, start_tm, end_tm, amount, cash, remark):

        dataBase = DataBase()
        sql = "insert into tb_order(cid, name, otype, order_tm, start_tm, end_tm, amount, cash, remark, insert_tm) " \
              "values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        param = (cId, name, otype, order_tm, start_tm, end_tm, amount, cash, remark, get_cur_time())

        bRet, sRet = dataBase.insert_data(sql, param)
        return bRet, sRet


    @staticmethod
    def update_node(oId, otype, order_tm, start_tm, end_tm, amount, cash, status, remark):

        dataBase = DataBase()
        sql = "update tb_order set otype=%s, order_tm=%s, start_tm=%s, end_tm=%s, amount=%s, cash=%s, " \
              "status = %s, remark=%s where oid= %s"
        param = (otype, order_tm, start_tm, end_tm, amount, cash, status, remark, oId)

        bRet, sRet = dataBase.update_data(sql, param)
        return bRet, sRet


    @staticmethod
    def update_node_status(oId, status):
        dataBase = DataBase()
        sql = "update tb_order set status = %s where oid = %s"
        param = (status, oId)

        bRet, sRet = dataBase.update_data(sql, param)
        if not bRet:
            return False, sRet

        return True, sRet


    @staticmethod
    def query_node_list(offset, limit, status, search):
        dataBase = DataBase()
        sql = "select * from tb_order where 1=1 "
        param = []
        if status and status != 'all':
            sql += "and status = %s "
            param.append(status)
        if search:
            search = "%%%s%%" % (search)
            sql += "and (cid like %s or name like %s)"
            param.append(search)
            param.append(search)

        sql += "order by oid desc limit %s, %s"
        param.append(offset)
        param.append(limit)

        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet

        return True, sRet


    @staticmethod
    def query_node_count(status=None, search=None):
        dataBase = DataBase()
        sql = "select count(*) as cnt from tb_order where 1=1 "
        param = []

        if status and status != 'all':
            sql += "and status = %s"
            param.append(status)
        if search:
            search = "%%%s%%" % (search)
            sql += "and (cid like %s or name like %s)"
            param.append(search)
            param.append(search)

        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
        if len(sRet) !=1:
            return True, 0

        return True, sRet[0]['cnt']

    @staticmethod
    def query_node_by_oid(oId):
        dataBase = DataBase()
        sql = "select * from tb_order where oid = %s"
        param = (oId, )

        bRet, sRet = dataBase.query_data(sql, param)
        if (not bRet) or (len(sRet) != 1):
            return False, sRet
        
        return True, sRet[0]


    @staticmethod
    def query_node_by_status(status):
        dataBase = DataBase()
        sql = "select count(*) as cnt from tb_order where 1=1 "
        param = []

        if status == 'all':
            pass
        else:
            sql += 'and status = %s'
            param.append(status)
        
        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
        if len(sRet) != 1:
            return True, 0
        
        return True, sRet[0]['cnt']

        
    @staticmethod
    def query_node_by_days(days):
        dataBase = DataBase()
        sql = "select oid, cid, name, date_format(end_tm, '%%Y-%%m-%%d') as end_tm from tb_order " \
              "where end_tm >= curdate() and end_tm < curdate() + %s"
        param = (days, )

        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
        
        return True, sRet

    
    @staticmethod
    def query_node_by_date(status, tg_date):
        dataBase = DataBase()
        sql = "select * from tb_order where status = %s and start_tm <= %s and %s <= end_tm"
        param = (status, tg_date, tg_date)

        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet

        return True, sRet
    @staticmethod
    def query_node_by_date2(status, tg_date):
        dataBase = DataBase()
        sql = "select * from tb_order where status = %s and start_tm <= %s and %s <= end_tm"
        param = (status, tg_date, tg_date)

        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
        if len(sRet) != 1:
            return True, 0
        
        return True, sRet[0]['cnt']


if __name__ == "__main__":
    print HjsOrderDao.query_node_by_status('stop')
    import datetime
    today = datetime.date.today()
    date_t=today.isoformat()
    #print HjsOrderDao.query_node_by_date('normal', '2018-10-11')
    print HjsOrderDao.query_node_by_date2('normal', date_t)
    d,x=HjsOrderDao.query_node_by_date('normal', date_t)
    #print len(x),x[0]['cnt']







[27/11/2018 17:11:15][DEBUG]../base/bs_database_pid.py::_exec_cmdstr(106): select count(*) as cnt from tb_order where 1=1 and status = %s	['stop']
[27/11/2018 17:11:15][DEBUG]../base/bs_database_pid.py::_new_conn(31): conn(host: 127.0.0.1, port: 3306, user: root, passwd: , db: Arthur_manage)
[27/11/2018 17:11:15][DEBUG]../base/bs_database_pid.py::_exec_cmdstr(106): select * from tb_order where status = %s and start_tm <= %s and %s <= end_tm	['normal', '2018-11-27', '2018-11-27']
[27/11/2018 17:11:15][DEBUG]../base/bs_database_pid.py::_new_conn(31): conn(host: 127.0.0.1, port: 3306, user: root, passwd: , db: Arthur_manage)
[27/11/2018 17:11:15][DEBUG]../base/bs_database_pid.py::_exec_cmdstr(106): select * from tb_order where status = %s and start_tm <= %s and %s <= end_tm	['normal', '2018-11-27', '2018-11-27']
[27/11/2018 17:11:15][DEBUG]../base/bs_database_pid.py::_new_conn(31): conn(host: 127.0.0.1, port: 3306, user: root, passwd: , db: Arthur_manage)


 (True, 0L)
(True, 0)


In [5]:
# -*- coding: utf-8 -*-

# author: s0nnet
# time: 2016-11-08
# desc: tb_user_dao


'''
# 用户操作
CREATE TABLE `tb_user` (
`uid` int(10) NOT NULL AUTO_INCREMENT,
`username` varchar(50) NOT NULL,
`nickname` varchar(50) NOT NULL,
`password` varchar(50) NOT NULL,
`phone` varchar(50) DEFAULT NULL,
`email` varchar(50) DEFAULT NULL,
`privilege` int(1) NOT NULL DEFAULT '1',
`remark` text,
PRIMARY KEY (`uid`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8;
'''

if __name__ == "__main__":
    import sys

    sys.path.append("..")
    sys.path.append("../base")

from hjs_cfg import *
from bs_util import *
from bs_database_pid import *


class HjsUserDao:
    
    @staticmethod
    def query_node_by_username(userName):
        dataBase = DataBase()
        sql = "select * from tb_user where username = %s"
        param = (userName, )

        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
        
        return True, sRet[0]


    @staticmethod
    def query_node_by_uid(uid):
        dataBase = DataBase()
        sql = "select * from tb_user where uid = %s"
        param = (uid, )

        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
        
        return True, sRet[0]

    
    @staticmethod
    def query_node_user_list():
        dataBase = DataBase()
        sql = "select * from tb_user"

        bRet, sRet = dataBase.query_data(sql, None)
        if not bRet:
            return False, sRet
        
        return True, sRet


    @staticmethod
    def insert_node_user(nickName, userName, passWord, Phone, Email, Priv):
        dataBase = DataBase()
        sql = "insert into tb_user(nickname, username, password, phone, email, privilege, lastlogin) values(%s, %s, %s, %s, %s, %s, %s)"
        param = (nickName, userName, passWord, Phone, Email, Priv, get_cur_time())

        bRet, sRet = dataBase.insert_data(sql, param)
        if not bRet:
            return False, sRet
        
        return True, sRet


    @staticmethod
    def update_node_user(uId, nickName, userName, passWord, Phone, Email, Priv):
        dataBase = DataBase()
        sql = "update tb_user set nickname = %s, username = %s, password = %s, phone = %s, email = %s, privilege= %s where uid = %s"
        param = (nickName, userName, passWord, Phone, Email, Priv, uId)

        bRet, sRet = dataBase.update_data(sql, param)
        if not bRet:
            return False, sRet
        
        return True, sRet


    @staticmethod
    def delete_node_user(uId):
        dataBase = DataBase()
        sql = "delete from tb_user where uid = %s"
        param = (uId, )

        bRet, sRet = dataBase.delete_data(sql, param)
        if not bRet:
            return False, sRet
        
        return True, sRet

    @staticmethod
    def update_lastlogin(uId):
        dataBase = DataBase()
        sql = "update tb_user set lastlogin = %s where uid = %s"
        param = (get_cur_time(), uId)
        
        return dataBase.update_data(sql, param)





if __name__ == "__main__":
    print HjsUserDao.query_node_by_username('leepand3')
    #print HjsUserDao.query_node_user_list()

    pass


[12/12/2018 16:23:15][DEBUG]../base/bs_database_pid.py::_exec_cmdstr(106): select * from tb_user where username = %s	['leepand3']
[12/12/2018 16:23:15][DEBUG]../base/bs_database_pid.py::_new_conn(31): conn(host: 127.0.0.1, port: 3306, user: root, passwd: , db: Arthur_manage)


(True, {'username': 'leepand3', 'uid': 31L, 'phone': '13380171631', 'lastlogin': datetime.datetime(2018, 12, 12, 16, 0, 44), 'privilege': 3L, 'password': '1233', 'nickname': 'leepand3', 'email': 'pandeng.li@163.com'})


In [2]:
# -*- coding: UTF-8 -*-
# author: leepand
# time: 2019-01-08
# desc: Algo 数据展示(my/other/zoo)

#update 2019-01-25

import datetime
from operator import itemgetter 
from Arthur.core.entities.bean.Arthur_service import ArthurService
from Arthur.core.entities.dao.Arthur_AlgoStar_dao import ArthurAlgoStarDao
#from Arthut.core.entities.dao.Arthur_serviceTags_dao import ArthurServTagsDao

class ArthurAlgoList:

    @staticmethod
    def _get_algolist_byuid(uId,service_status):
        #增加收藏的算法服务
        bRet,uid_algouid_list = ArthurAlgoStarDao.query_algouid_by_uidstar(uId)
        my_star_algo_list_all=[]
        my_star_algo_list=[]
        my_star_algo_list_dict=None
        if len(uid_algouid_list)>0:
            for algouid_algoname_dict in uid_algouid_list:
                algouid=algouid_algoname_dict['algouid']
                algoname = algouid_algoname_dict['algoname']
                #过滤掉自己收藏自己的算法，避免重复
                if int(algouid)-int(uId)==0:
                    continue
                bRet, my_star_algo_list_dict = ArthurService.service_user_list_filter(algouid,algoname,page=1,length=1000, status='normal', search=None)
                #print 'my_star_algo_list_dict',my_star_algo_list_dict
                if not bRet: my_star_algo_list = []
                if my_star_algo_list_dict is not None and "service_list" in my_star_algo_list_dict:
                    my_star_algo_list= my_star_algo_list_dict['service_list']
                else:
                    my_star_algo_list=[]
                my_star_algo_list_all.extend(my_star_algo_list)
    
        bRet, my_algo_list_dict = ArthurService.service_user_list(uId,page=1,length=1000, status=service_status, search=None)
        if not bRet: my_algo_list = []
        if my_algo_list_dict is not None and "service_list" in my_algo_list_dict:
            my_algo_list= my_algo_list_dict['service_list']
        else:
            my_algo_list=[]
        my_algo_list.extend(my_star_algo_list_all)
        my_algo_list_new=my_algo_list
        return  {
            "algo_list": my_algo_list_new
        }
    
    @staticmethod
    def _get_algolist_zoo():
        bRet, algo_list_zoo = ArthurService.service_zoo_list(page=1,length=1000, status='normal', search=None)
        if not bRet: algo_list_zoo = []
        if  algo_list_zoo is not None and "service_list" in algo_list_zoo:
            serviceList=algo_list_zoo['service_list']
            #serviceList_bystar = sorted(serviceList,key = itemgetter('star_cnt'),reverse = True)
            serviceList_bytime = sorted(serviceList,key = itemgetter('insert_tm'),reverse = True)

        else:
            serviceList=[]
            serviceList_bystar=[]
            serviceList_bytime=[]
        return {
                "algo_list": serviceList,#serviceList
                "algo_list_bytime":serviceList_bytime}
    @staticmethod
    def _get_tag_services(tag):
        bRet, tag_services_list = ArthurService.service_tag_list(search=tag)
        if not bRet: tag_services_list = []
        if  len(tag_services_list)>0:
            serviceList_bytime = sorted(tag_services_list,key = itemgetter('insert_tm'),reverse = True)

        else:
            tag_services_list=[]
            serviceList_bytime=[]
        return {
                "algo_list": tag_services_list,#serviceList
                "algo_list_bytime":serviceList_bytime}


    @staticmethod
    def data_show(uId,query_type,tag,service_status='normal'):
        if query_type=='zoo':
            if tag=="NonetagQuery":#myservice_layerui
                datas = {
                    "dt_algo_list":ArthurAlgoList._get_algolist_zoo()
                }
            else:
                datas = {
                    "dt_algo_list":ArthurAlgoList._get_tag_services(tag)
                }
        else:
            datas = {
                "dt_algo_list":ArthurAlgoList._get_algolist_byuid(uId,service_status)
            }
        return True, datas
    @staticmethod
    def algo_list_forautocomplete():
        bRet, sRet = ArthurAlgoList.data_show(1,'zoo')
        new_serv_list=[]
        try:
            for _inter in sRet['dt_algo_list']['algo_list']:
                new_serv_list.append(_inter['algoname'])
        except:
            pass
        return True,new_serv_list

if __name__ == "__main__":

    bRet, sRet = ArthurAlgoList.data_show(1,'user','NonetagQuery')

    print '>>>> ', sRet

    #print HjsIndex._get_expire_order()


[11/02/2019 20:03:12][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_exec_cmdstr(106): select * from tb_algo_star_thumb where uid=%s and star_status = %s	[1, 'star']
[11/02/2019 20:03:13][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_new_conn(31): conn(host: 127.0.0.1, port: 3306, user: root, passwd: , db: Arthur_manage)
[11/02/2019 20:03:13][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_exec_cmdstr(106): select count(*) as cnt from tb_algo where uid=%s and status = %s	[4L, 'normal']
[11/02/2019 20:03:13][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_new_conn(31): conn(host: 127.0.0.1, port: 3306, user: root, passwd: , db: Arthur_manage)
[11

>>>>  {'dt_algo_list': {'algo_list': [<Storage {'algoname': 'new_year_exp', 'status': 'normal', 'remark': '{"n":10}', 'insert_tm': '2019-02-09 12:02:55', 'uid': 1, 'atype': 'REST', 'token': '20bf7679146eaef99136cde84ccc1eba', 'field': 3L, 'algodesc': 'machine learning', 'host': '0.0.0.0', 'aid': 37, 'algo_tm': '2019-02-09', 'user_name': 'leepand6', 'port': 42756L}>, <Storage {'algoname': 'None', 'status': 'normal', 'remark': '\xe8\xaf\xb7\xe5\x88\xb0\xe4\xb8\xbb\xe9\xa1\xb5\xe7\xbc\x96\xe8\xbe\x91\xe6\xb7\xbb\xe5\x8a\xa0', 'insert_tm': '2019-01-29 08:27:11', 'uid': 1, 'atype': 'REST', 'token': '20bf7679146eaef99136cde84ccc1eba', 'field': 1L, 'algodesc': '\xe6\x9a\x82\xe6\x97\xa0\xe6\x8f\x8f\xe8\xbf\xb0\xef\xbc\x8c\xe8\xaf\xb7\xe5\x88\xb0\xe4\xb8\xbb\xe9\xa1\xb5\xe7\xbc\x96\xe8\xbe\x91\xe6\xb7\xbb\xe5\x8a\xa0', 'host': '0.0.0.0', 'aid': 35, 'algo_tm': '2019-01-29', 'user_name': 'leepand6', 'port': 48322L}>, <Storage {'algoname': 'new_publish_mokuai', 'status': 'normal', 'remark': '{n:10

In [3]:
bRet, sRet = ArthurAlgoList.data_show(1,'user','NonetagQuery')
sRet

[11/02/2019 20:03:26][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_exec_cmdstr(106): select * from tb_algo_star_thumb where uid=%s and star_status = %s	[1, 'star']
[11/02/2019 20:03:26][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_new_conn(31): conn(host: 127.0.0.1, port: 3306, user: root, passwd: , db: Arthur_manage)
[11/02/2019 20:03:26][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_exec_cmdstr(106): select count(*) as cnt from tb_algo where uid=%s and status = %s	[4L, 'normal']
[11/02/2019 20:03:26][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_new_conn(31): conn(host: 127.0.0.1, port: 3306, user: root, passwd: , db: Arthur_manage)
[11

{'dt_algo_list': {'algo_list': [<Storage {'algoname': 'new_year_exp', 'status': 'normal', 'remark': '{"n":10}', 'insert_tm': '2019-02-09 12:02:55', 'uid': 1, 'atype': 'REST', 'token': '20bf7679146eaef99136cde84ccc1eba', 'field': 3L, 'algodesc': 'machine learning', 'host': '0.0.0.0', 'aid': 37, 'algo_tm': '2019-02-09', 'user_name': 'leepand6', 'port': 42756L}>,
   <Storage {'algoname': 'None', 'status': 'normal', 'remark': '\xe8\xaf\xb7\xe5\x88\xb0\xe4\xb8\xbb\xe9\xa1\xb5\xe7\xbc\x96\xe8\xbe\x91\xe6\xb7\xbb\xe5\x8a\xa0', 'insert_tm': '2019-01-29 08:27:11', 'uid': 1, 'atype': 'REST', 'token': '20bf7679146eaef99136cde84ccc1eba', 'field': 1L, 'algodesc': '\xe6\x9a\x82\xe6\x97\xa0\xe6\x8f\x8f\xe8\xbf\xb0\xef\xbc\x8c\xe8\xaf\xb7\xe5\x88\xb0\xe4\xb8\xbb\xe9\xa1\xb5\xe7\xbc\x96\xe8\xbe\x91\xe6\xb7\xbb\xe5\x8a\xa0', 'host': '0.0.0.0', 'aid': 35, 'algo_tm': '2019-01-29', 'user_name': 'leepand6', 'port': 48322L}>,
   <Storage {'algoname': 'new_publish_mokuai', 'status': 'normal', 'remark': '{n:10

In [4]:
sRet

{'dt_algo_list': {'algo_list': [<Storage {'algoname': 'new_year_exp', 'status': 'normal', 'remark': '{"n":10}', 'insert_tm': '2019-02-09 12:02:55', 'uid': 1, 'atype': 'REST', 'token': '20bf7679146eaef99136cde84ccc1eba', 'field': 3L, 'algodesc': 'machine learning', 'host': '0.0.0.0', 'aid': 37, 'algo_tm': '2019-02-09', 'user_name': 'leepand6', 'port': 42756L}>,
   <Storage {'algoname': 'None', 'status': 'normal', 'remark': '\xe8\xaf\xb7\xe5\x88\xb0\xe4\xb8\xbb\xe9\xa1\xb5\xe7\xbc\x96\xe8\xbe\x91\xe6\xb7\xbb\xe5\x8a\xa0', 'insert_tm': '2019-01-29 08:27:11', 'uid': 1, 'atype': 'REST', 'token': '20bf7679146eaef99136cde84ccc1eba', 'field': 1L, 'algodesc': '\xe6\x9a\x82\xe6\x97\xa0\xe6\x8f\x8f\xe8\xbf\xb0\xef\xbc\x8c\xe8\xaf\xb7\xe5\x88\xb0\xe4\xb8\xbb\xe9\xa1\xb5\xe7\xbc\x96\xe8\xbe\x91\xe6\xb7\xbb\xe5\x8a\xa0', 'host': '0.0.0.0', 'aid': 35, 'algo_tm': '2019-01-29', 'user_name': 'leepand6', 'port': 48322L}>,
   <Storage {'algoname': 'new_publish_mokuai', 'status': 'normal', 'remark': '{n:10

In [6]:
sRet

{'dt_algo_list': {'algo_list': [<Storage {'algoname': 'new_year_exp', 'status': 'normal', 'remark': '{"n":10}', 'insert_tm': '2019-02-09 12:02:55', 'uid': 1, 'atype': 'REST', 'token': '20bf7679146eaef99136cde84ccc1eba', 'field': 3L, 'algodesc': 'machine learning', 'host': '0.0.0.0', 'aid': 37, 'algo_tm': '2019-02-09', 'user_name': 'leepand6', 'port': 42756L}>,
   <Storage {'algoname': 'None', 'status': 'normal', 'remark': '\xe8\xaf\xb7\xe5\x88\xb0\xe4\xb8\xbb\xe9\xa1\xb5\xe7\xbc\x96\xe8\xbe\x91\xe6\xb7\xbb\xe5\x8a\xa0', 'insert_tm': '2019-01-29 08:27:11', 'uid': 1, 'atype': 'REST', 'token': '20bf7679146eaef99136cde84ccc1eba', 'field': 1L, 'algodesc': '\xe6\x9a\x82\xe6\x97\xa0\xe6\x8f\x8f\xe8\xbf\xb0\xef\xbc\x8c\xe8\xaf\xb7\xe5\x88\xb0\xe4\xb8\xbb\xe9\xa1\xb5\xe7\xbc\x96\xe8\xbe\x91\xe6\xb7\xbb\xe5\x8a\xa0', 'host': '0.0.0.0', 'aid': 35, 'algo_tm': '2019-01-29', 'user_name': 'leepand6', 'port': 48322L}>,
   <Storage {'algoname': 'new_publish_mokuai', 'status': 'normal', 'remark': '{n:10

In [5]:
bRet, sRet = ArthurAlgoList.data_show(1,'user','NonetagQuery','all')


[11/02/2019 20:03:57][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_exec_cmdstr(106): select * from tb_algo_star_thumb where uid=%s and star_status = %s	[1, 'star']
[11/02/2019 20:03:57][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_new_conn(31): conn(host: 127.0.0.1, port: 3306, user: root, passwd: , db: Arthur_manage)
[11/02/2019 20:03:57][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_exec_cmdstr(106): select count(*) as cnt from tb_algo where uid=%s and status = %s	[4L, 'normal']
[11/02/2019 20:03:57][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_new_conn(31): conn(host: 127.0.0.1, port: 3306, user: root, passwd: , db: Arthur_manage)
[11

In [1]:
# -*- coding: utf-8 -*-
# author: leepand
# time: 2018-12-17
# desc: tb_algo_dao


'''
#算法管理表
DROP TABLE IF EXISTS `tb_algo`;
CREATE TABLE `tb_algo` (
`aid` int(10) NOT NULL AUTO_INCREMENT,
`uid` int(10) NOT NULL,
`algoname` varchar(50) NOT NULL,
`algodesc` varchar(50) NOT NULL,
`version` varchar(50) NOT NULL  DEFAULT 'v1.0.0',
`opertype` enum('publish','register')  DEFAULT 'publish',
`token` varchar(50) NOT NULL,
`pyfile` varchar(50) NOT NULL,
`funcslist` text NOT NULL,
`tags` text NOT NULL,
`field` int(10) NOT NULL,
`host` varchar(50) NOT NULL  DEFAULT '0.0.0.0',
`port` int(10) NOT NULL,
`atype` varchar(10) NOT NULL DEFAULT 'REST',
`algo_tm` datetime NOT NULL,
`status` enum('normal','stop','notpublish') DEFAULT 'notpublish',
`is_email` varchar(10) NOT NULL DEFAULT 'no',
`remark` text NOT NULL,
`insert_tm` datetime NOT NULL,
PRIMARY KEY (`aid`),
KEY `uid` (`uid`),
CONSTRAINT `tb_algo_ibfk_1` FOREIGN KEY (`uid`) REFERENCES `tb_user` (`uid`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8;
'''


#update 2019-01-25
from Arthur.core.entities.arthur_web_cfg import *
from Arthur.core.entities.base.bs_util import *
from Arthur.core.entities.base.bs_database_pid import DataBase
from Arthur.core.entities.base.bs_time import get_cur_day,get_cur_time

class ArthurAlgoDao:

    @staticmethod
    def insert_node(uId, algoname,algodesc,opertype,token,pyfile,funcslist,tags,field,host,port,atype, algo_tm, is_email,remark):

        dataBase = DataBase()
        sql = "insert into tb_algo(uid, algoname, algodesc,opertype,token,pyfile,funcslist,tags,field,host,port,atype, algo_tm, is_email,remark, insert_tm) " \
              "values(%s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s,%s, %s,%s,%s)"
        param = (uId, algoname,algodesc,opertype,token,pyfile,funcslist,tags,field,host,port,atype, algo_tm,is_email, remark, get_cur_time())

        bRet, sRet = dataBase.insert_data(sql, param)
        return bRet, sRet


    @staticmethod
    def update_node(aId, atype, algo_tm,status, remark):

        dataBase = DataBase()
        sql = "update tb_algo set atype=%s, algo_tm=%s, " \
              "status = %s, remark=%s where aid= %s"
        param = (aotype, algo_tm, status, remark, aId)

        bRet, sRet = dataBase.update_data(sql, param)
        return bRet, sRet


    @staticmethod
    def update_node_status(aId, status):
        dataBase = DataBase()
        updatetime=get_cur_day(0, format="%Y-%m-%d %H:%M:%S")
        #sql = "update tb_order set status = %s where oid = %s"
        #param = (status, oId)
        sql = "update tb_algo set status = %s,insert_tm = %s where aid = %s"
        param = (status,updatetime,aId)
        bRet, sRet = dataBase.update_data(sql, param)
        if not bRet:
            return False, sRet

        return True, sRet


    @staticmethod
    def query_node_list(offset, limit, status, search):
        dataBase = DataBase()
        sql = "select * from tb_algo where 1=1 "
        param = []
        if status and status != 'all':
            sql += "and status = %s "
            param.append(status)
        if search:
            search = "%%%s%%" % (search)
            sql += "and (uid like %s or algoname like %s)"
            param.append(search)
            param.append(search)

        sql += "order by aid desc limit %s, %s"
        param.append(offset)
        param.append(limit)

        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet

        return True, sRet


    @staticmethod
    def query_node_count(status=None, search=None):
        dataBase = DataBase()
        sql = "select count(*) as cnt from tb_algo where 1=1 "
        param = []

        if status and status != 'all':
            sql += "and status = %s"
            param.append(status)
        if search:
            search = "%%%s%%" % (search)
            sql += "and (uid like %s or algoname like %s)"
            param.append(search)
            param.append(search)

        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
        if len(sRet) !=1:
            return True, 0

        return True, sRet[0]['cnt']
    #用户的服务list查询
    @staticmethod
    def query_node_list_by_uid2(uId,offset, limit, status, search):
        dataBase = DataBase()
        sql = "select * from tb_algo where uid=%s "
        param = [uId]
        if status and status != 'all':
            sql += "and status = %s "
            param.append(status)
        if search:
            search = "%%%s%%" % (search)
            sql += "and (uid like %s or algoname like %s)"
            param.append(search)
            param.append(search)

        sql += "order by aid desc limit %s, %s"
        param.append(offset)
        param.append(limit)

        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet

        return True, sRet
    #用户的服务list查询 update 2019-02-19/原查询效率太低
    @staticmethod
    def query_node_list_by_uid(uId,status='all', search=None):
        dataBase = DataBase()
        sql = "select * from tb_algo where uid=%s "
        param = [uId]
        if status and status != 'all':
            sql += "and status = %s "
            param.append(status)

        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet

        return True, sRet
    @staticmethod
    def query_node_count_by_uid(uId,status=None, search=None):
        dataBase = DataBase()
        sql = "select count(*) as cnt from tb_algo where uid=%s "
        param = [uId]

        if status and status != 'all':
            sql += "and status = %s"
            param.append(status)
        if search:
            search = "%%%s%%" % (search)
            sql += "and (uid like %s or algoname like %s)"
            param.append(search)
            param.append(search)

        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
        if len(sRet) !=1:
            return True, 0

        return True, sRet[0]['cnt']
    #aid的服务list查询,服务于发布
    @staticmethod
    def query_node_by_aid(aId):
        dataBase = DataBase()
        sql = "select * from tb_algo where aid = %s"
        param = (aId, )

        bRet, sRet = dataBase.query_data(sql, param)
        if (not bRet) or (len(sRet) != 1):
            return False, sRet
        
        return True, sRet[0]
    #aid的服务info查询,服务于服务详情页
    @staticmethod
    def query_node_by_proj_uid(uId,AlgoName):
        dataBase = DataBase()
        sql = "select * from tb_algo where uid = %s and algoname = %s"
        param = (uId, AlgoName)

        bRet, sRet = dataBase.query_data(sql, param)
        if (not bRet) or (len(sRet) != 1):
            return False, sRet
        
        return True, sRet[0]

    @staticmethod
    def query_node_by_status(status):
        dataBase = DataBase()
        sql = "select count(*) as cnt from tb_order where 1=1 "
        param = []

        if status == 'all':
            pass
        else:
            sql += 'and status = %s'
            param.append(status)
        
        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
        if len(sRet) != 1:
            return True, 0
        
        return True, sRet[0]['cnt']

        
    @staticmethod
    def query_node_by_days(days):
        dataBase = DataBase()
        sql = "select oid, cid, name, date_format(order_tm, '%%Y-%%m-%%d') as end_tm from tb_order " \
              "where order_tm >= curdate() and order_tm < curdate() + %s"
        param = (days, )

        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
        
        return True, sRet

    
    @staticmethod
    def query_node_by_date(status, tg_date):
        dataBase = DataBase()
        #sql = "select * from tb_algo where status = %s and algo_tm <= %s and algo_tm >= %s"
        sql = "select * from tb_algo where status = %s and insert_tm <= %s and insert_tm >= %s"
        sql_stop = "select * from tb_algo where status = %s and insert_tm <= %s and insert_tm >= %s"
        param = (status,tg_date,get_cur_day(0, format="%Y-%m-%d"))
        if status=='normal':
            bRet, sRet = dataBase.query_data(sql, param)
        else:
            bRet, sRet = dataBase.query_data(sql_stop, param)
        if not bRet:
            return False, sRet

        return True, sRet

    @staticmethod
    def query_node_cnt_by_date(status, tg_date):
        dataBase = DataBase()
        sql = "select count(*) as cnt from tb_algo where status = %s and algo_tm <= %s and algo_tm >= %s"
        sql_stop = "select count(*) as cnt from tb_algo where status = %s and insert_tm <= %s and insert_tm >= %s"
        param = (status, tg_date,get_cur_day(0, format="%Y-%m-%d"))

        if status=='normal':
            bRet, sRet = dataBase.query_data(sql, param)
        elif status=='stop':
            bRet, sRet = dataBase.query_data(sql_stop, param)
        else:
            bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
        if len(sRet) != 1:
            return True, 0
        
        return True, sRet[0]['cnt']

if __name__ == "__main__":
    #print HjsOrderDao.query_node_by_status('stop')
    import datetime
    today = datetime.date.today()
    date_t=today.isoformat()
    uId=41
    algoname='leepand'
    pyfile='./predict.py'
    funcslist='predict,pre2'
    algodesc='text mining'
    tags='nlp'
    token='sdksadfksdf'
    field=41
    atype='recom'
    host='0.0.0.0'
    is_email='no'
    port=9000
    algo_tm= "2018-03-12 12:23:11"
    #order_tm=date_t
    
    remark='just test'
    #ArthurAlgoDao.update_node_status(1,'normal')
    ArthurAlgoDao.query_node_list_by_uid(1)
    #print HjsOrderDao.insert_node(cId, name, otype, order_tm, remark)
    #print HjsOrderDao.query_node_by_date2('normal', '2019-01-02')
    #ArthurAlgoDao.insert_node(uId, algoname,algodesc,token,pyfile,funcslist,tags,field,host,port,atype, algo_tm, is_email,remark)

#uId, algoname,algodesc,token,pyfile,funcslist,tags,field,host,port,atype, algo_tm, is_email,remark

#uId, algoname,algodesc,token,pyfile,funcslist,tags,host,port,atype, algo_tm, is_email,remark

In [6]:
_,d=ArthurAlgoDao.query_node_list_by_uid(1)


In [3]:
import time 
t1=time.time()
_,d=ArthurAlgoDao.query_node_list_by_uid(1,status='all')

print time.time()-t1

0.0337710380554
